In [ ]:
!pip install scikit-multilearn
!pip install -U sentence-transformers
!pip install --upgrade google-api-python-client
# !pip install google-colab

from google.colab import drive
drive.mount('/content/drive')
from mlsmote import get_minority_instace, MLSMOTE

from sentence_transformers import SentenceTransformer, InputExample, losses, LoggingHandler, util
from torch.utils.data import DataLoader, Dataset
from sklearn.utils import resample,shuffle
from imblearn.over_sampling import SMOTE 
from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from skmultilearn.model_selection import iterative_train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.adapt import MLkNN
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import ClassifierChain

from sklearn.model_selection import learning_curve
from matplotlib import pyplot as plt
import plotly.express as px
from sklearn import metrics
import pandas as pd
import numpy as np
import json
import torch


from sentence_transformers.datasets import SentenceLabelDataset
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import TripletEvaluator
from datetime import datetime
from werkzeug.datastructures import ImmutableMultiDict


import logging
import os
import random
from collections import defaultdict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 1.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.0 MB/s 
     |████████████████████████████████| 5.8 MB 24.8 MB/s 
     |████████████████████████████████| 1.3 MB 56.0 MB/s 
     |████████████████████████████████| 182 kB 54.0 MB/s 
     |████████████████████████████████| 7.6 MB 17.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=85c22db359e068e691a2dcf42899542e1543c695edd3bf5fac2a453855cc0d61
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████

In [ ]:
!sudo apt-get install python3.10

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10-minimal
Suggested packages:
  python3.10-venv binfmt-support
The following NEW packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10 python3.10-minimal
0 upgraded, 4 newly installed, 0 to remove and 7 not upgraded.
Need to get 5,098 kB of archives.
After this operation, 19.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.10-minimal amd64 3.10.8-1+bionic1 [823 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.10-minimal amd64 3.10.8-1+bionic1 [1,970 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bioni

##Fine Tune model

Dataloader


```
DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, *, prefetch_factor=2,
           persistent_workers=False)
```
```dataset``` = datasets that can be accessed like this dataset[idx] or like this iter(dataset)

```sampler``` = **only** for map style datasets, sampler classes are sued to specify the sequence of indices/keys used in data loading. a sampler could randomly permute a list of indices and yield each one at a time, or yield a small number of them for mini-batch SGD.

```batch_sampler``` = a custom sampler that yields a list of batch indices at a time

**Automatic batching can be triggered via ```batch_size``` and ```drop_last```**

```batch_size``` != None yields batched samples instead of individual sampels

In [ ]:
class MyDataset(Dataset):
 
  def __init__(self,file_name):
    data=pd.read_csv(file_name).reset_index()
    data = data.drop(columns = ['level_0' , 'Unnamed: 0', 'index'])
 
    x=data['text'].values
    y=data[['Operations', 'Service', 'Business', 'Software', 'Electrical',
       'Mechanical', 'Data Analytics']].values

    print(type(x))
 
    # self.x_train=torch.tensor(x,dtype=torch.string)
    self.x_train=x
    self.y_train=y
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]
    # label = self.y[idx]
    # text = self.x[idx]
    # sample = {"Text": text, "Class": label}
    # return sample


def flatten(l):
    return [item for sublist in l for item in sublist]



logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

# Inspired from torchnlp
def ticket_dataset():
    train_set = train_examples
    dev_set = val_examples
    test_set = test_examples

    # For dev & test set, we return triplets (anchor, positive, negative)
    random.seed(42) #Fix seed, so that we always get the same triplets
    dev_triplets = triplets_from_labeled_dataset(dev_set)
    test_triplets = triplets_from_labeled_dataset(test_set)


    return train_set, dev_triplets, test_triplets


def triplets_from_labeled_dataset(input_examples):
    # Create triplets for a [(label, sentence), (label, sentence)...] dataset
    # by using each example as an anchor and selecting randomly a
    # positive instance with the same label and a negative instance with a different label
    triplets = []
    # print(input_examples)
    label2sentence = defaultdict(list)
    for inp_example in input_examples:
        label2sentence[inp_example.label].append(inp_example)

    for inp_example in input_examples:
        anchor = inp_example
    
        if len(label2sentence[inp_example.label]) < 2: #We need at least 2 examples per label to create a triples
            continue
        # print(label2sentence[inp_example.label])
        positive = None
        while positive is None or positive.guid == anchor.guid:
            positive = random.choice(label2sentence[inp_example.label])
            # print('anchor', type(anchor.label))
            # print('positive', type(positive.label))

        negative = None
        while negative is None or negative.label == anchor.label:
            negative = random.choice(input_examples)

        triplets.append(InputExample(texts=[anchor.texts[0], positive.texts[0], negative.texts[0]]))

    return triplets


Prepare for Dataloader in InputExample format

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/translated_data(3).csv').reset_index()
data = data.sample(frac=1).reset_index(drop=True)

n = 80
data = data.head(int(len(data)*(n/100)))
n=20
tmp = data.tail(int(len(data)*(n/100)))
val_data = tmp.head(int(len(data)/2))
test_data = tmp.tail(int(len(data)/2))

train_examples = [[InputExample(guid = random.randint(1,30), texts=[i], label=1) for i in data['text'].loc[data['Operations']==1].values],
                  [InputExample(guid = random.randint(31,60), texts=[i], label=2) for i in data['text'].loc[data['Service']==1].values],
                  [InputExample(guid = random.randint(61,90), texts=[i], label=3) for i in data['text'].loc[data['Business']==1].values],
                  [InputExample(guid = random.randint(91,120), texts=[i], label=4) for i in data['text'].loc[data['Software']==1].values],
                  [InputExample(guid = random.randint(121,150),texts=[i], label=5) for i in data['text'].loc[data['Electrical']==1].values],
                  [InputExample(guid = random.randint(151,180), texts=[i], label=6) for i in data['text'].loc[data['Mechanical']==1].values],
                  [InputExample(guid = random.randint(181,200),texts=[i], label=7) for i in data['text'].loc[data['Data Analytics']==1].values]]
train_examples= flatten(train_examples)

val_examples = [[InputExample(guid = random.randint(1,30), texts=[i], label=1) for i in val_data['text'].loc[val_data['Operations']==1].values],
                  [InputExample(guid = random.randint(31,60), texts=[i], label=2) for i in val_data['text'].loc[val_data['Service']==1].values],
                  [InputExample(guid = random.randint(61,90), texts=[i], label=3) for i in val_data['text'].loc[val_data['Business']==1].values],
                  [InputExample(guid = random.randint(91,120), texts=[i], label=4) for i in val_data['text'].loc[val_data['Software']==1].values],
                  [InputExample(guid = random.randint(121,150), texts=[i], label=5) for i in val_data['text'].loc[val_data['Electrical']==1].values],
                  [InputExample(guid = random.randint(151,180), texts=[i], label=6) for i in val_data['text'].loc[val_data['Mechanical']==1].values],
                  [InputExample(guid = random.randint(181,200), texts=[i], label=7) for i in val_data['text'].loc[val_data['Data Analytics']==1].values]]
val_examples= flatten(val_examples)

test_examples = [[InputExample(guid = random.randint(1,30), texts=[i], label=1) for i in test_data['text'].loc[test_data['Operations']==1].values],
                  [InputExample(guid = random.randint(31,60), texts=[i], label=2) for i in test_data['text'].loc[test_data['Service']==1].values],
                  [InputExample(guid = random.randint(61,90), texts=[i], label=3) for i in test_data['text'].loc[test_data['Business']==1].values],
                  [InputExample(guid = random.randint(91,120), texts=[i], label=4) for i in test_data['text'].loc[test_data['Software']==1].values],
                  [InputExample(guid = random.randint(121,150), texts=[i], label=5) for i in test_data['text'].loc[test_data['Electrical']==1].values],
                  [InputExample(guid = random.randint(151,180), texts=[i], label=6) for i in test_data['text'].loc[test_data['Mechanical']==1].values],
                  [InputExample(guid = random.randint(181,200), texts=[i], label=7) for i in test_data['text'].loc[test_data['Data Analytics']==1].values]]
test_examples= flatten(test_examples)


Create a torch.DataLoader that passes training batch instances to our model

In [ ]:
# You can specify any huggingface/transformers pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-base
model_name = 'all-MiniLM-L6-v2'

train_batch_size = 32
output_path = (
    "output/finetune-batch-hard-trec-"
    + model_name
    + "-"
    + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)
num_epochs = 10

logging.info("Loading dataset")
train_set, dev_set, test_set = ticket_dataset()

Load pretrained model

In [ ]:

logging.info("Load model")
transformer_model = SentenceTransformer(model_name)

"""We create a special dataset "SentenceLabelDataset" to wrap out train_set
. It will yield batches that contain at least two samples with the same label
"""

train_data_sampler = SentenceLabelDataset(train_set)
train_dataloader = DataLoader(train_data_sampler, batch_size=32, drop_last=True)

"""# Load pretrained model"""

logging.info("Load model")
transformer_model = SentenceTransformer(model_name)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Triplet Losses

In [ ]:
### There are 4 triplet loss variants:
### - BatchHardTripletLoss
### - BatchHardSoftMarginTripletLoss
### - BatchSemiHardTripletLoss
### - BatchAllTripletLoss
#######################################

train_loss = losses.BatchAllTripletLoss(model=transformer_model)
# train_loss = losses.BatchHardTripletLoss(model=model)
# train_loss = losses.BatchHardSoftMarginTripletLoss(model=model)
# train_loss = losses.BatchSemiHardTripletLoss(model=model)

logging.info("Read TREC val dataset")
dev_evaluator = TripletEvaluator.from_input_examples(dev_set, name='trec-dev')

logging.info("Performance before fine-tuning:")
dev_evaluator(transformer_model)

warmup_steps = int(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data

Train model

In [ ]:
transformer_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    save_best_model = True,
    output_path=output_path,
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Load the stored model and evaluate its performance on dataset


In [ ]:
logging.info("Evaluating model on test set")
test_evaluator = TripletEvaluator.from_input_examples(test_set, name='trec-test')
transformer_model.evaluate(test_evaluator)

0.59375

##Multi-label classification

Prepare data for multi-lable classification


In [ ]:
sentences = data['text'].values
X = transformer_model.encode(sentences)

Y = data.drop(columns = ['text', 'level_0' , 'index'])
Y = Y.fillna(0)

Data for testing in ImmutableMultiDict format

In [ ]:
input = [ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'Hello! We are very happy to be using LIFTBOT'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')]),
         ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'The gripper is broken and we need somone to repair it'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')]),
         ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'The remote says: group up but its stuck'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')]),
         ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'When will LIFTBOT arrive on site'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')]),
         ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'Could I have the daily report'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')]),
         ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'Robot doesnt move'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')]),
         ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'The robot dropped'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')]),
         ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'We cannot close the platform door'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')]),
         ImmutableMultiDict(
        [('SmsMessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('NumMedia', '0'),
        ('ProfileName', 'Alex'),
        ('SmsSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('WaId', '4917662094803'),
        ('SmsStatus', 'received'),
        ('Body', 'Here is an image of the robot'),
        ('To', 'whatsapp:+3197010253713'),
        ('NumSegments', '1'),
        ('ReferralNumMedia', '0'),
        ('MessageSid', 'SMa4375112d3018c3b4da9c3498b65b3a7'),
        ('AccountSid', 'AC72edff6111ea7fde1809ac64a3a7d3f7'),
        ('From', 'whatsapp:+4917662094803'),
        ('ApiVersion', '2010-04-01')])
]

X_input = []
for i in input:
  X_input.append(i.getlist('Body'))

X_test = flatten(X_input)
X_test = transformer_model.encode(X_test)
# print(X_test.shape)
# print(type(X_test))

Apply TSNE to see distance betwen vectors and whther they cluster using linear seperation


In [ ]:
#force X into 2 dimensions so we can plot it
# pca = PCA(n_components = 2) 
# X_graph = pca.fit_transform(X) 
# X_graph.shape

X_embedded = TSNE(n_components=2, perplexity = 40, learning_rate=500).fit_transform(X)
# textx = get_text()
for i in Y.values:
  i = str(i)

df_embeddings = pd.DataFrame(X_embedded)
df_embeddings = df_embeddings.rename(columns={0:'x',1:'y'})
# df_embeddings = df_embeddings.assign(label=Y.values)
df_embeddings = df_embeddings.assign(text='text '+sentences)

fig = px.scatter(
    df_embeddings, x='x', y='y',
    # color='text', labels={'text': 'y'},
    hover_data=['text'], title = 'Embedding Visualization')

fig.show()

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.



Create synthetic data for minority labels using MLSMOTE

In [ ]:
X = pd.DataFrame(X)
print(X.columns)
print(Y.columns)
X_sub, y_sub = get_minority_instace(X, Y)   #Getting minority instance of that datframe
X_res,y_res =MLSMOTE(X_sub, y_sub, 30)

#concatinate synthtic data with already labeled data
X = pd.concat((X,X_res), ignore_index = True)
Y = pd.concat((Y,y_res), ignore_index = True)

Train Test split

In [ ]:
X_train, y_train, X_val, y_val = iterative_train_test_split(X.values, Y.values, test_size=0.2)

Apply dimensionality reduction to embeddings

In [ ]:
pca = PCA(n_components = 10) 
X_train = pca.fit_transform(X_train) 

In [ ]:
import pickle as pk 

pk.dump(pca, open("pca.pkl","wb"))

In [ ]:

X_val = pca.transform(X_val) 
X_test = pca.transform(X_test)



Choose a base classifier to train on

In [ ]:
# model = LogisticRegression(random_state=0, class_weight='balanced')
# model = LogisticRegression()
# model = SVC(C = 1, kernel = 'rbf', gamma = 1)
model = RandomForestClassifier(criterion = 'gini', n_estimators = 40)

One-vs-Rest classification result using Jaccard Score, F1 Macro and Hamming Loss


In [ ]:
y_train = y_train.astype(bool)

clf = OneVsRestClassifier(model)
clf.fit(X_train, y_train)

Y_pred_ovr = clf.predict(X_val)

ovr_jaccard_score = metrics.jaccard_score(y_val, Y_pred_ovr, average="samples", labels = np.unique(Y_pred_ovr))
ovr_f1_macro = metrics.f1_score(y_val, Y_pred_ovr, average="macro")
ovr_hamming = metrics.hamming_loss(y_val, Y_pred_ovr)
print('jaccard score: ', ovr_jaccard_score)
print('f1 macro score: ', ovr_f1_macro)
print('Hamming loss: ', ovr_hamming)

Classifier chain result using Jaccard Score, F1 Macro and Hamming Loss

In [ ]:
import pickle
chains = [ClassifierChain(model, order="random", random_state=i) for i in range(10)]
# pickle.dump(chains, open("chains.pkl","wb"))
for chain in chains:
    chain.fit(X_train, y_train)

pickle.dump(chains, open("chains.pkl","wb"))

Y_pred_chains = np.array([chain.predict(X_val) for chain in chains])
chain_jaccard_scores = [
    metrics.jaccard_score(y_val, Y_pred_chain >= 0.5, average="samples")
    for Y_pred_chain in Y_pred_chains
]
chain_f1_macro = [
    metrics.f1_score(y_val, Y_pred_chain >= 0.5, average="macro")
    for Y_pred_chain in Y_pred_chains
]
chain_ham_loss = [
    metrics.hamming_loss(y_val, Y_pred_chain >= 0.5)
    for Y_pred_chain in Y_pred_chains
]

print('chain jaccard score: ', chain_jaccard_scores)
print('chain f1 macro score: ', chain_f1_macro)
print('chain hamming loss: ', chain_ham_loss)

Ensemble of Classifier chains result using Jaccard Score, F1 Macro and Hamming Loss

In [ ]:
Y_pred_ensemble = Y_pred_chains.mean(axis=0)

ensemble_jaccard_score = metrics.jaccard_score(
    y_val, Y_pred_ensemble >= 0.5, average="samples"
)
ensemble_f1_macro = metrics.f1_score(
    y_val, Y_pred_ensemble >= 0.5, average="macro"
)
ensemble_ham_loss = metrics.hamming_loss(
    y_val, Y_pred_ensemble >= 0.5)

print('ensemble jaccard score: ', ensemble_jaccard_score)
print('ensemble f1 macro score: ', ensemble_f1_macro)
print('ensemble hamming loss: ', ensemble_ham_loss)

Graph representation of results

In [ ]:
model_scores = [ovr_jaccard_score] + chain_jaccard_scores
print(len(model_scores))
model_scores.append(ensemble_jaccard_score)
model_scores = np.array(model_scores)

model_names = (
    "Independent",
    "Chain 1",
    "Chain 2",
    "Chain 3",
    "Chain 4",
    "Chain 5",
    "Chain 6",
    "Chain 7",
    "Chain 8",
    "Chain 9",
    "Chain 10",
    # "Chain 1",
    # "Chain 2",
    # "Chain 3",
    # "Chain 4",
    # "Chain 5",
    # "Chain 6",
    # "Chain 7",
    # "Chain 8",
    # "Chain 9",
    # "Chain 10",
    "Ensemble",
)

x_pos = np.arange(len(model_names))
print(x_pos.shape)
print(model_scores.shape)

# Plot the Jaccard similarity scores for the independent model, each of the
# chains, and the ensemble (note that the vertical axis on this plot does
# not begin at 0).

fig, ax = plt.subplots(figsize=(7, 4))
ax.grid(True)
ax.set_title("Support Vector Machine Classifier Chain Ensemble")
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, rotation="vertical")
ax.set_ylabel("Jaccard Similarity Score")
ax.set_ylim([min(model_scores) * 0.9, max(model_scores) * 1.1])
colors = ["r"] + ["b"] * len(chain_jaccard_scores) + ["g"]
ax.bar(x_pos, model_scores, alpha=0.5, color=colors)
plt.tight_layout()
plt.show()

##Demo

In [ ]:
Y_pred_chains = np.array([chain.predict(X_test) for chain in chains])
Y_pred_ensemble = Y_pred_chains.mean(axis=0)
Y_pred_ensemble = (Y_pred_ensemble>=0.5).astype(int)

print(Y_pred_ensemble)

list_of_labels = Y.columns.to_list()
# list_of_labels = ['Dima', 'Heitor', 'Artem', '']
for i in range(len(X_input)):
  print("Incomming message from client: ", X_input[i])
  for j in range(len(Y_pred_ensemble[i])):
    if Y_pred_ensemble[i][j]==1:
      print("Ticket assigned to: ", list_of_labels[j])
  print('\n')